# Linear Regression in Python 
#### Data Analytics with High Performance Computing

This brief practical is intended to provide you with a starting point to experiment with some aspects of linear regression. It's also worth trying to see whether you can convert some of the content from lectures that is expressed into mathematical notation into code.

As you evaluate the cells below, try to make sure you understand what's happening at each stage. I encourage you to tweak the code to experiment with what happens if you do things slightly differently (e.g. if you swap Series with slices from DataFrames, etc.).

## Simple Linear Regression

We'll start by looking at the familiar 2D case (where $y$ is a function of a single variable $x$).

First, import the necessary packages:


In [ ]:
import pandas as pd #pandas to give us dataframes
import seaborn as sns #seaborn for some prettier plots
import numpy as np #used later for matrix manipulation

import matplotlib.pyplot as plt # We'll use matplotlib to create some plots of the data

Load the data from `cats.csv` into a DataFrame called `cats`. This dataet contains the body weight and heart weight of cats as seen in the lecture.

In [ ]:
cats=pd.read_csv("cats.csv")
cats.head()

Let's plot the body weight against the heart weight. What so we see? Is there a relationship between the two?

In [ ]:
plt.scatter(cats["Bwt"],cats["Hwt"])

Let's see how we can fit a linear regression model using **scikit learn**.

In [ ]:
from sklearn.linear_model import LinearRegression

We'll store our result in `reg`:

In [ ]:
reg=LinearRegression().fit(cats[['Bwt']],cats[['Hwt']])

We can obtain the coefficients of the mdoelusing the `reg` object:

In [ ]:
print(reg.intercept_)
print(reg.coef_)

As a brief aside, we can also see the arguments which `fit` was originally called with, as follows:

In [ ]:
reg.get_params()

Let's manually get a predicted value of $y$ for each $x$ point using the normal formula for a straight line, $y=\theta_0+ \theta_1 x$ where $y$ corresponds to the *predicted* value for `Hwt`, $x$ is the `Bwt`.

In [ ]:
ypred=reg.intercept_ + cats[['Bwt']]*reg.coef_
ypred

We'll know plot the original data, and superimpose the straight line that comes from `ypred`:

In [ ]:
plt.scatter(cats["Bwt"],cats["Hwt"])
plt.plot(cats["Bwt"],ypred["Bwt"], color='red')

Instead of evaluating the formula itself, you can use the built in `predict` method, which gives the same results:

In [ ]:
#ypred2=reg.predict(cats[['Bwt']])
#ypred2

In [ ]:
plt.scatter(cats['Bwt'],cats['Hwt'])
plt.plot(cats['Bwt'],ypred2, color='green')

So far, we've allowed LinearRegression.fit() to find our straight line that best fits the data. To convince yourself that what you're plotting is a straigh line with the calculated slope and intercept, try changing the values of a and b below, recalcuating ymanual and replotting the graph.

In [ ]:
theta0=-0.5
theta1=4
ymanual= theta0+theta1*cats[['Bwt']]

In [ ]:
plt.scatter(cats['Bwt'],cats['Hwt'])
plt.plot(cats['Bwt'],ypred['Bwt'], color='red')
plt.plot(cats['Bwt'],ymanual['Bwt'], color='green')

Later on in this practical, we're going to see how you can find the values of $\theta_0$ and $\theta_1$ that give the best fit from first principles, instead of using `fit()` from `sklearn`.

### Calculating the Cost Function

You will remember that Linear Regression works by minimising a **cost function** based on the **Mean Squared Error**. 

The cost function is calculated repeatedly in the process of fitting (or, to use Machine Learning terminology, in learning the model).

Let's calculate the Mean Square Error here. We're looking at a simple case here where the target variable, $y$, depends on only one variable, $x$.

Our model can be expressed as $ h_\theta(x) = \theta_0+\theta_1 x $ and the records in the dataset above can be expressed as the set $\{(x^{(i)},y^{(i)})\}$ where $i$ runs from $1$ to $m$. Also, our cost functio is given by:

$$ J(\theta_0, \theta_1) = \frac{1}{m}\sum_{i=1}^m (h_\theta(x^{(i)})-y^{(i)})^2$$

**TASK**: Write some code which calculates this quantity for the dataset above. Remember $h_\theta(x)$ is the function for the straight line. I'm not asking here for code to *minimise* this function, just to *calculate* it given your straight line (defined by your parameters $\mathbf{\theta}$) and the original data points.

**Hint:** Depending on your approach, you may need to convert a pandas dataframe to a numpy array. If you need to do this you can use `to_numpy()`.

You should get the answer `2.08009128`.



In [ ]:
theta1= reg.coef_
theta0= reg.intercept_

#answer
h=theta0+theta1*cats[["Bwt"]]
cost_function=np.sum((h["Bwt"]-cats["Hwt"])**2)/144

print(cost_function)


As an aside, you can also calculate the mean square error using sklearn. Note that I said that cost function above was *based* on the mean squared error.

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(cats["Hwt"],ypred)

## Some quick exploratory data analysis

We are now going to look at the public dataset *mtcars*. This data was extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973-74 models). Import the mtcars dataset using the code below

In [ ]:
# Import CSV mtcars
mtcars=pd.io.parsers.read_csv("mtcars.csv",index_col=0) # this function helps us preserve the rownames of the dataset as row names instead of assigning them to a new column
mtcars.head()

Recall that we can view basic descriptive statistics of this dataset using the `describe` and `info` method:

In [ ]:
mtcars.describe()

In [ ]:
print(mtcars.shape)

print("=====")

mtcars.info()

It's often informative to visualise data like this with a scatterplot matrix to get a quick impression of the data before exploring in more detail.

In [ ]:
pd.plotting.scatter_matrix(mtcars)

Seaborn provides a version that is a bit prettier out-of-the-box. This is a good way to roughly determine if there is any linear correlation between any pair of variables. 

In [ ]:
sns.pairplot(mtcars)

You can subset the plot to those variables that seem to have a linear relationship/are correlated as follows:

In [ ]:
sns.pairplot(mtcars[['mpg','wt','disp']])

Look at the correlation between these 3 variables in order to better undesratnd any association.

In [ ]:
#Pearson's correlation coefficient for mpg and disp
print(mtcars['mpg'].corr(mtcars['disp']))

print("========")

#Correlation matrix for mpg, disp, wt
print(mtcars[['mpg','disp', 'wt']].corr())

## Multiple linear regression with Scikit Learn

Now we will explore linear regression using more than one variable.

While simple linear regression gives a model corresponding a straight **line**, linear regression with two independent variables gives a model corresponding to a flat **plane** (and in general, in higher dimensions, a so-called hyperplane). The "height" of the plane corresponds to the value of the hypothesis function which defines the model.

We're going to continue to work with the subset of the mtcars dataset that we looked at above, and in particular we will try to predict the number of miles a car can drive for each gallon of petrol (`mpg`) using the weight (`wt` in lb/1000) of the car and its engine displacement (`disp` in cu.in.). Let's visualise the resulting plane below:


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter( mtcars['wt'], mtcars['disp'],mtcars['mpg'], c='blue', s=60)

# create a grid of values in the wt (p1) and disp (p2) directions
# to help us draw a plane
p1, p2 = np.mgrid[1:6, 0:500:100] 

theta0=30 
theta1=-3.3
theta2=-0.015


plane= theta0 + theta1*p1 + theta2*p2

ax.plot_surface(p1,p2,plane,color='red',alpha=0.7)
plt.show()

Play around with the values of $\theta_0, \theta_1, \theta_2$ and see how this changes the fitted plane in the plot above

Now, lets find the optimal parameters for this model (those that give the "best fit") without using library functions but using the analytical solution provided by the normal equuations, $\boldsymbol{\theta}=(X^T X)^{-1} X^T y$

We will now try to use the matrix version of the normal equation (as described in lectures) to find the optimal parameters for this model. The matrix version requires two matrices X and y defined as:

$$ X =
 \begin{pmatrix}
  1 & x_1^{(1)} & x_2^{(1)} & \cdots & x_n^{(1)} \\
  1 & x_1^{(2)} & x_2^{(2)} & \cdots & x_n^{(2)} \\
  \vdots  & \vdots & \vdots  & \ddots & \vdots  \\
  1 & x_1^{(m)} & x_2^{(m)} & \cdots & x_n^{(m)} \\
\end{pmatrix}, y = 
\begin{pmatrix}
  y^{(1)} \\
  y^{(2)} \\
  \vdots \\
  y^{(m)}
\end{pmatrix} $$

Let's create the matrices $\boldsymbol{y}$, $\boldsymbol{X}$ 

In [ ]:
#insert your code here

X=mtcars[['wt','disp']]
X["col1"]=1
print(X.head())

# bring itno right order
X=X[["col1","wt","disp"]]
print(X.head())

y=mtcars['mpg']


We are now ready to solve for the parameters using the equation:

$$
\mathbf{\theta}=(X^TX)^{-1}X^Ty
$$

**TASK**: Build up this equation in Python. We _could_ convert things back to NumPy at the start (and for high performance with larger data, this might be desirable) but you can do some things like the transpose with Pandas direclty. The inverse, on the other hand, is easier to do in NumPy with `np.linalg.inv()`.

Here are some hints:

1. Both dataframes and numpy arrays have a transpose method e.g. `M.transpose()`
2. Matrix multiplication can be performed with both dataframes and numpy arrays using the dot method, e.g. `A.dot(B)`

In [ ]:
#indert your code here

theta=np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
theta

**TASK**: Now see if you can use these coefficients to predict the mpg for the Volvo 142E using the weight and displacement figures shown above for that vehicle.

Hint:

- You can use matrix multiplication to multiply a vector containing the required values by the co-efficients matrix. Recall that,  $\hat{y}=X \hat{\theta}$.

The predicted value for the Volvo 142E should be 23.50057 miles per gallon.

In [ ]:
#indert your code here

X.loc['Volvo 142E'].dot(theta)

## Multiple linear regression using StatsModels and built in functions

We have been doing things mostly "by hand" above to make sure you understand a little bit about how linear regression is implemented.

In practice, we usually use existing methods, as long as we know exctly what these do and calculate, and the same functions used above for single variable linear regression also work for multiple variable regression.

The library StatsModels comes with a variety of measures already calculated and easily accessible as well. Let's import the library and start fiting our regression model.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
#using scikit learn
reg_1=LinearRegression().fit(mtcars[['wt','disp']],mtcars['mpg'])

# using statsModels
reg_2=smf.ols('mpg ~ wt + disp', data=mtcars).fit()

You can look at the coefficent estimates as follows:

In [ ]:
reg_2.params

In [ ]:
print(reg_1.intercept_)
print(reg_1.coef_)

Look at the documentation for the model object returned to see some of the other information that's returned in the object

https://www.statsmodels.org/stable/regression.html

https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html

We can automatically extract the fitted values using `reg2.fittedvalues`:

In [ ]:
reg_2.fittedvalues.head()

`reg_2.resid` contains the residuals, i.e., the differences between true and fitted values. Recall the noise $\epsilon$ we talked about in the lecture.

In [ ]:
reg_2.resid.head()

Use this to calculate the sum of the squared errors of the model.

In [ ]:
#answer

np.sum(reg_2.resid**2)
#or reg_2.ssr

Look at the Adjusted R squared, `reg2.rsquared_adj`,  and do some plots to assess the model fit and whether certain assumptions of the model hold. Some of the things to look for:

- Normality of the residuals (look at a hstpgram of the residuals)
- Uncorrelated residuals   (look at an autocorrelation (ACF) plot of the residuals, e.g., pd.plotting.autocorrelation_plot())
- Plot of fitted values vs actual values
- Plot of fitted values vs Residuals, looking for constant variance of the residuals, i.e. is there any pattern in the residuals?

In [ ]:
# insert your code here

plt.hist(reg_2.resid)

pd.plotting.autocorrelation_plot(reg_2.resid)

plt.scatter(reg2.fittedvalues, mtcars['mpg'])

plt.scatter(reg2.fittedvalues,reg2.resid)


Do you know what an autocorrelation is and how to interpret an autocorrelation plot?

Using the function `summary` on the object `reg2` you can get more information about the model.

In [ ]:
print(reg2.summary())

What information does this table give you?

The top left column gives you information about the dependent variable, the method use dfor fitting, number of observations used, and the degrees of freedom of the model. 

The right column prints out out some key statistics such as $R^2$, adjusted $R^2$, log likelihood value and the $F$-statistic along with its associated $p$-value, Prob($F$-Statistic). The $F$-statistic and corresponding $p$-value are used to assess whethewr all the coefficients of the model are jointly equal to zero against the hypotheis that at least one is different than zero. If the reported $p$-value is greater than your predefined significance level then there is not sufficient evidence to support the model. The significanc elevel we usually use is $0.05$, i.e., $5\%$.

Below these we get all the information we need regarding the coefficients estimates, their standard error (std err) and at the end a $95\%$ confidence interval for each coefficient estimate.  The column named $t$ holds a t-statistic for each coefficient estimate ($t$-value = coefficient estimate/ Standard error) in order to perform a statistical hypothesis test for each coefficent. These hypotheses tests, assess whether there is evidence that a coefficent can be zero given that all other explanatory variables/features are included in the model.

For each $t$-value, a $p$-value is reported in the column P>|t|. If this value is higher than your defined significance level then there is not sufficient evidence that the corresponding coefficient is different than zero given that all other variables are included in the model. Similar conclusions we can draw by looking at the confidence interval of each coefficient estimate and see if it includes 0.


## Polynomial Regression

Polynomial regression can be used to fit non-linear models to data. It uses linear regression and simply adds polynomial terms to the set of feature values.

Consider the following data set of tortoise shell length and clutch size (number of eggs):

In [ ]:
tortoise=pd.read_csv('tortoise.csv', skipinitialspace=True)

In [ ]:
tortoise.plot(kind="scatter", x='length', y='clutch', colormap=None)

_Data source: Ashton, K.G., R.L. Burke and J.N. Layne. 2007. Geographic variation in body and clutch size of gopher tortoises. Copeia 2007: 355-363._

It doesn't look like there is an obvious linear fit to this data but let's try to use linear regression to predict clutch size from length and plot the data with the fitted model:

In [ ]:
fit_1=smf.ols('clutch ~ length', data=tortoise).fit()

In [ ]:
from matplotlib import pyplot as plt
tortoise.plot(kind='scatter', x='length', y='clutch')
plt.plot(tortoise.length,fit_1.fittedvalues, 'r')

Does this look like a great fit to the data? Calculate the sum of square errors, it should be $186.152$, print and evaluate its statistics. What can we conclude from the R-squared statistic?

In [ ]:
# answer here

# calculate sum of square errors
print(np.sum(fit_1.resid**2))
#or fit_1.ssr

# summary statistics
print(fit_1.summary())


# Here are a number of different approaches to calculate what we refer to as the mean squared error:
##print(mean_squared_error(tortoise.clutch, fit_1.fittedvalues))
##print(mean_squared_error(tortoise.clutch, ypred))
##J=0
##for i in range(len(tortoise)):  
##    J=J+(tortoise.clutch.iloc[i]-fit_1.fittedvalues.iloc[i])**2
    
##J=J/(len(tortoise))
##print(J)
##print(fit_1.ssr/len(tortoise))
##print(fit_1.mse_resid/18*16)



#reg=LinearRegression().fit(tortoise.iloc[:,0:1],tortoise.iloc[:,1:2])
#ypred=reg.predict(tortoise.iloc[:,0:1])
#


Can we do better with polynomial regression without seriously overfitting the model by adding the square of the length as an additional feature to the feature set? Create an extra column in the dataframe that holds lengthSq, i.e. length squared, and fit a new model, fit_2, including this in. 

In [ ]:
# insert your code here

tortoise['lengthSq']= tortoise['length']**2
fit_2=smf.ols('clutch ~ length + lengthSq', data=tortoise).fit()


Plot the fitted values onto the graph with lines between them to quickly show the shape of the new polynomial model:

In [ ]:
from matplotlib import pyplot as plt
tortoise.plot(kind='scatter', x='length', y='clutch', colormap=None)
plt.plot(tortoise.length,fit_1.fittedvalues, 'r')
plt.plot(tortoise.length,fit_2.fittedvalues, 'g')

The model seem's to capture the data better and reflects that very large tortoises tend to lay fewer eggs than middle sized tortoises. Use `summary()` to evaluate its statistics. What can we conclude from the R-squared statistic, compared to the previous model?

In [ ]:
print(fit_2.summary())